In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import os, urllib, itertools, shutil, random
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
import PIL
import torch
from pathlib import Path

In [ ]:
# !unzip Poetry.zip -d myfiles

In [ ]:
!unzip poems.csv.zip -d myfiles

Archive:  poems.csv.zip
  inflating: myfiles/poems.csv       


In [ ]:
mydf = pd.read_csv("/content/myfiles/poems.csv")

In [ ]:
mydf["text"]

,text
0,NaN
1,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...
2,Пилигрим\nАллах ли там среди пустыни\nЗастывши...
3,"О, не скрывай! Ты плакала об нем –\nИ я его лю..."
4,"Ты знал ли дикий край, под знойными лучами,\nГ..."
...,...
19311,"Хорошо, что в этом мире\nЕсть магические ночи,..."
19312,Пробочка над крепким иодом!\nКак ты скоро пере...
19313,"Друзья, друзья! Быть может, скоро —\nИ не во с..."
19314,"Увы, дитя! Душе неутоленной\nНе снишься ль ты ..."


In [ ]:
import os
output_folder = 'output_files'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for index, row in mydf.iterrows():
    filename = f'{index}.txt'
    filepath = os.path.join(output_folder, filename)

    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(str(row['text']))

print(f"Файлы успешно сохранены в папке '{output_folder}'")

Файлы успешно сохранены в папке 'output_files'


In [ ]:

from nltk.tokenize import WordPunctTokenizer
from transformers import AutoTokenizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import string
import json

In [ ]:
MODES = ['train', 'val', 'test']

In [ ]:
#TRAIN_DIR = Path('/content/myfiles/Poetry/')
TRAIN_DIR = Path('/content/output_files')
files_list = list(TRAIN_DIR.rglob('*.txt'))

In [ ]:
class PoetryDataset(Dataset):
  def __init__(self, files, mode):
    super().__init__()
    self.files = files
    self.mode = mode
    self.tokenizer = WordPunctTokenizer()

    if not os.path.exists("word2vec_model.model"):
      self.train_word2vec()
    with open('word_to_index_dict.json', 'r') as f:
      loaded_dict = json.load(f)
      self.word_to_index_dict = loaded_dict


  def __len__(self):
    return len(self.files)

  def loadtxt(self, file):
    with open(file, 'r', encoding='utf-8') as text:
      content = text.read()
    return content


  def preprocess_for_w2vec(self, content):
    lines = content.split('\n')
    tokens = []
    for line in lines:
      if len(line) > 1:
        line_tokens = self.tokenizer.tokenize(line.lower())
        line_tokens = [token for token in line_tokens if token not in string.punctuation] # под сомнением
        tokens.extend(line_tokens)

        # if len(lines) > 0 and line != lines[-1]:
        #   tokens.append("EOL") # end of line token

    return tokens


  def train_word2vec(self):
    sentences = []
    all_tokens_set = set()
    texts = [self.loadtxt(file) for file in self.files]
    for text in texts:
      sentence_of_text = self.preprocess_for_w2vec(text)
      all_tokens_set.update(sentence_of_text)
      sentences.append(sentence_of_text)

    model_w2v = Word2Vec(sentences=sentences,
                       min_count=1,
                       vector_size=50,
                       window=5,
                       workers=4)

    self.word_to_index_dict = {token: i for i, token in enumerate(all_tokens_set)}
    self.word_to_index_dict["PAD"] = len(self.word_to_index_dict)
    with open('word_to_index_dict.json', 'w') as f:
      json.dump(self.word_to_index_dict, f)
    model_w2v.save("word2vec_model.model")

    return model_w2v




  def __getitem__(self, idx):
      fileee = self.loadtxt(self.files[idx])
      sequence_of_tokens = self.preprocess_for_w2vec(fileee)

      inputs = sequence_of_tokens[:-1]
      targets = sequence_of_tokens[1:]


      sequence_of_indices_input = [self.word_to_index_dict.get(token) for token in inputs]
      sequence_of_indices_target = [self.word_to_index_dict.get(target) for target in targets]

      return {"inputs": torch.tensor(sequence_of_indices_input),
            "targets": torch.tensor(sequence_of_indices_target)}

In [ ]:
train_val_files = list(TRAIN_DIR.rglob('*.txt'))
train_dataset = PoetryDataset(train_val_files, mode='train')

In [ ]:
with open('word_to_index_dict.json', 'r') as f:
    loaded_dict = json.load(f)

In [ ]:
len(loaded_dict)

255935

**ТЕПЕРЬ МОДЕЛЬКА**

In [ ]:
import torch.nn as nn

In [ ]:
class PoetryLSTM(nn.Module):
  def __init__(self):
    super(PoetryLSTM, self).__init__()
    input_size = 50
    hidden_size = 50
    vectorizer = Word2Vec.load("word2vec_model.model")
    vocab_size = len(loaded_dict)
    # предобученные в word2vec эмбеддинги
    embedding_matrix = np.zeros((vocab_size, input_size))
    for i in range(len(vectorizer.wv)):
      word = vectorizer.wv.index_to_key[i]
      if word in vectorizer.wv:
        embedding_matrix[i] = vectorizer.wv[word]

    self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32),freeze=False)

    self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=1,
                            batch_first=True,
                            dropout=0.2)

    self.fc_out = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
      h0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
      c0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)

      embedded_x = self.embedding(x)

      lstm_out, (new_hidden_state, new_cell_state) = self.lstm(embedded_x, (h0, c0))
      linear_layer_out_before_softmax = self.fc_out(lstm_out)

      return linear_layer_out_before_softmax


In [ ]:
model_ltsm_generator = PoetryLSTM()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
batch_size = 4
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_ltsm_generator.parameters(), lr=0.001)


print(device)
model_ltsm_generator.to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


cuda


PoetryLSTM(
  (embedding): Embedding(255935, 50)
  (lstm): LSTM(50, 50, batch_first=True, dropout=0.2)
  (fc_out): Linear(in_features=50, out_features=255935, bias=True)
)

**ТЕПЕРЬ DATALOADER + PADDING**

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def custom_collate(batch):
  max_seq_len = 50
  inputs_batch_list = [item["inputs"] for item in batch]
  targets_batch_list = [item["targets"] for item in batch]
  padded_inputs = pad_sequence(inputs_batch_list, batch_first = True, padding_value = loaded_dict["PAD"])
  padded_targets = pad_sequence(targets_batch_list, batch_first = True, padding_value = loaded_dict["PAD"])
  mask = (padded_targets != loaded_dict["PAD"]).float()
  return {"inputs": padded_inputs[:, :max_seq_len],
          "targets": padded_targets[:, :max_seq_len],
          "mask": mask}

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, collate_fn = custom_collate)

In [ ]:
# проверка на размерности

it = iter(train_loader)
batch = next(it)
print(batch["inputs"].size())
print(batch["targets"].size())

torch.Size([4, 50])
torch.Size([4, 50])


In [ ]:
len(loaded_dict)

255935

In [ ]:
!nvidia-smi

Tue Feb 18 18:50:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             29W /   70W |   10216MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
for epoch in range(num_epochs):
    model_ltsm_generator.train()
    total_loss = 0

    for batch in train_loader:
        inputs = batch["inputs"].to(device).long()
        targets = batch["targets"].to(device)
        mask = batch["mask"].to(device)  # Маска для игнорирования паддинга

        optimizer.zero_grad()

        output_probs = model_ltsm_generator(inputs)
        output_probs_flat = output_probs.view(-1, output_probs.shape[-1])
        targets_flat = targets.view(-1).long()

        loss = criterion(output_probs_flat, targets_flat)
        loss = (loss * mask.view(-1)).mean()  # Уч итываем маску

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

Epoch 1, Loss: 4.678112339642656
Epoch 2, Loss: 4.3351950260096395
Epoch 3, Loss: 4.134919483781626
Epoch 4, Loss: 3.95751977825935
Epoch 5, Loss: 3.8114296854295255
Epoch 6, Loss: 3.675982125453708
Epoch 7, Loss: 3.536171718077226
Epoch 8, Loss: 3.4180729217375254
Epoch 9, Loss: 3.2994628667485815
Epoch 10, Loss: 3.2004805562362386


In [ ]:
index_to_word = {value: key for key, value in loaded_dict.items()}

In [ ]:
len(index_to_word)

255935

In [ ]:
index_to_word[136]

'чудак'

In [ ]:
def generate_text(model, start_sequence, max_length, loaded_dict, index_to_word):
    """
    Генерирует текст на основе начальной последовательности.

    """

    input_indices = [loaded_dict.get(token) for token in start_sequence]


    input_tensor = torch.tensor(input_indices).unsqueeze(0).to(device)

    generated_text = list(start_sequence)  # Список сгенерированных слов

    # return input_tensor
    model.to(device)
    model.eval()

    for i in range(max_length):
        output_linear = model(input_tensor)
        # return output_linear[:, -1, :]
        output_linear_flat = output_probs.view(-1, output_probs.shape[-1])
        output_probs_last_step = torch.softmax(output_linear[:, -1, :], dim=-1)

        top_k_indises = torch.topk(output_probs_last_step[0], k=25).indices

        predicted_index = random.choice(top_k_indises)

        predicted_word = index_to_word[predicted_index.item()]

        generated_text.append(predicted_word)

        input_tensor = torch.cat((input_tensor.to(device), torch.tensor([predicted_index]).unsqueeze(0).to(device)), dim=1).to(device)

    return ' '.join(generated_text)


In [ ]:
start_sequnce_example = ["любви", "надежды", "тихой", "славы"]
generated_poem = generate_text(model_ltsm_generator, start_sequnce_example, max_length=100, loaded_dict=loaded_dict, index_to_word=index_to_word)

print(generated_poem)

любви надежды тихой славы хочу тебя мне дороже судьбе меня всегда хочу быть может о ты ни что за ты хочешь любить о всех чуде мне хочется меня с людьми ли вечно друг мой удел ли за нас он ты меня но о ты мне больно всё былое уж нет нет их души любви их страстей а если тебя когда же мне мое не зови ты любил мой голос так не знает но всё же не зови любви души моей когда много раз в огне моей тебе зачем всегда за мною душа моя как сердце ты ль с собою милой на тебя под кровом


любви надежды тихой славы дух своей тобой во мглу ищу когда она ли ее вдохновенья и ныне люблю я знаю за меня к иному душа любовь любовью тобой и к тебе готов ли верь зачем меня — она мой нежный — лишь взор из дальних дорог там — мы любим я весь вас для счастья тебя как и каждый миг на устах меня от сна … свет как то мы был только солнце он как странно все и тихо из сердца чтоб — не верь ль не друг как — как то к сердцу же дано !.. всё в тот и то это давно

[ ]


****